Optuna can optimize hyper-parameter.

I use LightGBM and Optuna.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import sklearn
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

from lightgbm import LGBMClassifier
from lightgbm import plot_importance

import optuna.integration.lightgbm as lgb
import optuna

# Import Data.

In [ ]:
train_data = pd.read_csv('/kaggle/input/tabular-playground-series-nov-2021/train.csv')
test_data = pd.read_csv('/kaggle/input/tabular-playground-series-nov-2021/test.csv')

ID is not necessary.

So I'm deleting it.

In [ ]:
train_df = train_data.drop(['id'], axis=1)
test_df = test_data.drop(['id'], axis=1)

In [ ]:
X = train_df.drop('target', axis=1)
y = train_df['target']

In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
_test = scaler.fit_transform(test_df)

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, shuffle=True, random_state=266632)

In [ ]:
Train_Data = lgb.Dataset(X_train, y_train)
Valid_Data = lgb.Dataset(X_valid, y_valid)

# Optuna's Setup

In [ ]:
def objective(trial):
    param = {
        'metric': 'auc',
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'feature_pre_filter': False,
        # 'device':'gpu',
    }
    model = lgb.train(param,
                    Train_Data,
                    valid_sets=Valid_Data,
                    early_stopping_rounds=15,
                    verbose_eval = False)

    preds = model.predict(X_valid)
    pred_labels = np.rint(preds)
    accuracy = sklearn.metrics.accuracy_score(y_valid, pred_labels)
    return accuracy

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=75)

In [ ]:
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

We get optimized hyper-parameter by Optuna.

Let's use this.

In [ ]:
params = {'metric': 'auc',
          # 'device':'gpu',  # If you don't use GPU, Please Comment out this.
          'early_stopping_rounds': 15}
params = params.update(study.best_trial.params)

# Train and Predict.

## Train

In [ ]:
model = train(params,
              Train_Data,
              valid_sets=Valid_Data,
              early_stopping_rounds=15)

## Predict

In [ ]:
prediction = model.predict(_test)

# Submit

In [ ]:
submission = pd.DataFrame({'id':test_data['id'], 'target': prediction})

In [ ]:
submission.to_csv('submission.csv', index=False)